# 10 - DTCAI: Distance-to-Crash AI

DTCAI (Lee, Jeong, Park & Ahn, 2025) addresses a fundamental weakness of LPPLS:
**most LPPLS fits are unreliable**. The method trains a classifier on the 7
LPPLS parameters to predict whether a fit is "reliable" (predicted tc within
10 days of an actual crash). The final score is:

$$\text{DTCAI} = \text{DTC} \times P(\text{reliable})$$

where DTC (Distance-to-Crash) = $(t_2 - t_1) / (t_c - t_1)$ measures how
close the current time is to the predicted critical time.

This notebook:
1. Generates a labeled dataset from BTC price history
2. Trains three classifier variants (ANN, Random Forest, Logistic Regression)
3. Evaluates classifier accuracy and compares model types
4. Runs DTCAI predictions on historical episodes
5. Compares DTCAI vs. raw LPPLS confidence

> **Reference:** Lee, G., Jeong, M., Park, T. & Ahn, K. (2025).
> "More Than Ex-Post Fitting: LPPL and Its AI-Based Classification."
> *Humanities and Social Sciences Communications*, 12, 236.

> **DISCLAIMER:** This software is for academic research and educational purposes only.
> It does not constitute financial advice.

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fatcrash.data.ingest import from_sample
from fatcrash.data.transforms import log_prices, time_index
from fatcrash.nn.dtcai_data import (
    generate_dtcai_dataset,
    FEATURE_NAMES,
)
from fatcrash.nn.dtcai import (
    train_dtcai,
    predict_dtcai,
    compute_dtc,
)
from fatcrash.nn.crash_labels import detect_crashes
from fatcrash.indicators.lppls_indicator import compute_confidence

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (14, 5)

## 1. The DTCAI pipeline

The key insight: raw LPPLS fits produce many false positives. Instead of
trusting every fit, we ask: "does this set of fitted parameters *look like*
parameters that historically preceded actual crashes?"

```
Price series
  --> Rolling LPPLS fits (multiple seeds per window)
    --> Extract 7 parameters: (A, B, C, tc, phi, omega, beta)
      --> Label each fit: reliable (tc near actual crash) vs unreliable
        --> Train classifier on (params -> reliable?)
          --> At prediction time: DTCAI = DTC * P(reliable)
```

The 7 features are:
- **A, B**: Linear LPPLS parameters (level and trend)
- **C**: Oscillation amplitude = sqrt(C1^2 + C2^2)
- **tc**: Predicted critical time
- **phi**: Oscillation phase = atan2(C2, C1)
- **omega**: Log-periodic frequency
- **beta**: Power-law exponent (= m in standard LPPLS notation)

## 2. Generate labeled dataset

We run LPPLS fits across rolling windows on BTC history, then label each
fit using the Bree & Joseph (2013) crash criterion.

In [ ]:
df_btc = from_sample("btc")
prices = df_btc["close"].values

print(f"BTC: {len(prices)} daily observations")
print(f"Date range: {df_btc.index[0].date()} to {df_btc.index[-1].date()}")

# Detect actual crashes using Bree & Joseph criterion
crashes = detect_crashes(prices, lookback=262, forward=60, threshold=0.25)
print(f"\nDetected {len(crashes)} crash events:")
for c in crashes:
    peak_date = df_btc.index[c.peak_date_idx].date()
    trough_date = df_btc.index[c.trough_date_idx].date()
    print(f"  Peak {peak_date}, trough {trough_date}, drawdown {c.drawdown:.1%}")

In [ ]:
# Generate labeled LPPLS parameter dataset
# This runs many LPPLS fits -- takes a few minutes
print("Generating DTCAI dataset (rolling LPPLS fits)...")
dataset = generate_dtcai_dataset(
    prices,
    window_size=504,   # ~2 years
    step_size=21,      # ~1 month between windows
    n_fits_per_window=5,  # 5 fits per window (faster for demo)
    crash_lookback=262,
    crash_forward=60,
    crash_threshold=0.25,
    tc_tolerance=10,
    seed=42,
)

print(f"\nDataset: {len(dataset.X)} labeled LPPLS fits")
print(f"  Reliable (label=1): {dataset.y.sum()}")
print(f"  Unreliable (label=0): {(dataset.y == 0).sum()}")
print(f"  Reliable fraction: {dataset.y.mean():.1%}")
print(f"  Features: {dataset.feature_names}")

## 3. Explore the feature space

Visualize how reliable vs. unreliable fits differ in parameter space.

In [ ]:
if len(dataset.X) > 0:
    df_feat = pd.DataFrame(dataset.X, columns=FEATURE_NAMES)
    df_feat["reliable"] = dataset.y

    # Summary statistics by label
    print("Feature statistics by label:")
    print()
    display(df_feat.groupby("reliable")[FEATURE_NAMES].mean().T.round(3))
else:
    print("No data generated -- check dataset generation above.")

In [ ]:
if len(dataset.X) > 0:
    # Pairwise scatter: omega vs beta colored by label
    fig, axes = plt.subplots(1, 3, figsize=(16, 4))

    pairs = [("omega", "beta"), ("tc", "C"), ("A", "B")]
    for ax, (fx, fy) in zip(axes, pairs):
        mask_0 = dataset.y == 0
        mask_1 = dataset.y == 1
        ax.scatter(df_feat.loc[mask_0, fx], df_feat.loc[mask_0, fy],
                   c="gray", alpha=0.3, s=10, label="Unreliable")
        ax.scatter(df_feat.loc[mask_1, fx], df_feat.loc[mask_1, fy],
                   c="red", alpha=0.6, s=20, label="Reliable")
        ax.set_xlabel(fx)
        ax.set_ylabel(fy)
        ax.legend(fontsize=8)

    fig.suptitle("LPPLS Parameter Space: Reliable vs Unreliable Fits", fontsize=13)
    plt.tight_layout()
    plt.show()

## 4. Train classifiers

We train three model types from the paper:
- **ANN**: 4-layer neural network (Input(7) -> 256 -> 128 -> 64 -> 1)
- **RF**: Random Forest (100 trees)
- **LogReg**: Logistic Regression baseline

All use random oversampling to handle class imbalance (most fits are unreliable).

In [ ]:
models = {}

for model_type in ["RF", "LogReg"]:
    print(f"\nTraining {model_type}...")
    try:
        m = train_dtcai(dataset, model_type=model_type, seed=42)
        models[model_type] = m
        print(f"  {model_type} trained successfully.")
    except ImportError as e:
        print(f"  Skipped: {e}")

# Try ANN (requires PyTorch)
print("\nTraining ANN...")
try:
    m = train_dtcai(dataset, model_type="ANN", epochs=50, lr=0.01, seed=42)
    models["ANN"] = m
    print("  ANN trained successfully.")
except ImportError as e:
    print(f"  Skipped (PyTorch not installed): {e}")

print(f"\nTrained models: {list(models.keys())}")

## 5. Evaluate classifiers

Quick evaluation: run predictions on the training data and compute accuracy.
(For a proper evaluation, use the accuracy report which does train/test splits.)

In [ ]:
from fatcrash.nn.dtcai import _predict_reliability

eval_results = []

for name, model in models.items():
    preds = []
    for i in range(len(dataset.X)):
        prob = _predict_reliability(model, dataset.X[i])
        preds.append(prob)
    preds = np.array(preds)

    # Binary predictions at 0.5 threshold
    y_pred = (preds >= 0.5).astype(int)
    y_true = dataset.y

    acc = (y_pred == y_true).mean()
    # Precision/recall for the reliable class
    tp = ((y_pred == 1) & (y_true == 1)).sum()
    fp = ((y_pred == 1) & (y_true == 0)).sum()
    fn = ((y_pred == 0) & (y_true == 1)).sum()
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0.0

    eval_results.append({
        "model": name,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
    })
    print(f"{name:>8s}: acc={acc:.3f}, prec={prec:.3f}, rec={rec:.3f}, F1={f1:.3f}")

eval_df = pd.DataFrame(eval_results)
eval_df

In [ ]:
if len(eval_df) > 0:
    fig, ax = plt.subplots(figsize=(8, 4))
    x = np.arange(len(eval_df))
    width = 0.2

    for i, metric in enumerate(["precision", "recall", "f1"]):
        ax.bar(x + i * width, eval_df[metric], width, label=metric.title())

    ax.set_xticks(x + width)
    ax.set_xticklabels(eval_df["model"])
    ax.set_ylabel("Score")
    ax.set_ylim(0, 1)
    ax.set_title("DTCAI Classifier Comparison")
    ax.legend()
    plt.tight_layout()
    plt.show()

## 6. Run DTCAI predictions on historical windows

Use the best model to compute DTCAI scores across rolling windows
of BTC history. Compare with raw LPPLS confidence.

In [ ]:
# Pick the best available model
best_name = eval_df.loc[eval_df["f1"].idxmax(), "model"] if len(eval_df) > 0 else None
print(f"Using model: {best_name}")

if best_name:
    best_model = models[best_name]

    # Compute LPPLS confidence for comparison
    lp = log_prices(df_btc)
    t = time_index(df_btc)
    conf_arr, tc_mean_arr, tc_std_arr = compute_confidence(
        t, lp, n_windows=30, n_candidates=20,
    )

    # Compute rolling DTCAI scores
    window_size = 504
    step = 21
    log_p = np.log(np.maximum(prices, 1e-10))

    dtcai_dates = []
    dtcai_scores = []
    dtc_scores = []
    reliability_scores = []

    for start in range(0, len(prices) - window_size, step):
        end = start + window_size
        window_t = np.arange(start, end, dtype=np.float64)
        window_lp = log_p[start:end]

        try:
            result = predict_dtcai(best_model, window_t, window_lp)
            dtcai_dates.append(df_btc.index[end - 1])
            dtcai_scores.append(result.dtcai)
            dtc_scores.append(result.dtc)
            reliability_scores.append(result.reliability)
        except Exception:
            dtcai_dates.append(df_btc.index[end - 1])
            dtcai_scores.append(np.nan)
            dtc_scores.append(np.nan)
            reliability_scores.append(np.nan)

    dtcai_dates = pd.DatetimeIndex(dtcai_dates)
    dtcai_scores = np.array(dtcai_scores)
    dtc_scores = np.array(dtc_scores)
    reliability_scores = np.array(reliability_scores)

    print(f"Computed {len(dtcai_scores)} DTCAI scores")
    valid = dtcai_scores[~np.isnan(dtcai_scores)]
    print(f"  Valid: {len(valid)}, mean={valid.mean():.3f}, max={valid.max():.3f}")

In [ ]:
if best_name:
    fig, axes = plt.subplots(4, 1, figsize=(14, 14), sharex=True)

    # Price
    axes[0].plot(df_btc.index, df_btc["close"], color="steelblue", linewidth=0.8)
    axes[0].set_yscale("log")
    axes[0].set_ylabel("Price (USD)")
    axes[0].set_title("BTC/USD Price")

    # Mark crash peaks
    for c in crashes:
        axes[0].axvline(df_btc.index[c.peak_date_idx], color="red",
                        linestyle="--", alpha=0.5)

    # LPPLS confidence
    axes[1].plot(df_btc.index[:len(conf_arr)], conf_arr,
                 color="blue", linewidth=0.8, alpha=0.7)
    axes[1].set_ylabel("LPPLS Confidence")
    axes[1].set_title("Raw LPPLS Confidence")
    axes[1].set_ylim(0, 1)
    for c in crashes:
        axes[1].axvline(df_btc.index[c.peak_date_idx], color="red",
                        linestyle="--", alpha=0.5)

    # DTCAI score
    axes[2].plot(dtcai_dates, dtcai_scores, color="darkred",
                 linewidth=1.0, alpha=0.8)
    axes[2].fill_between(dtcai_dates, 0, dtcai_scores,
                         where=np.array(dtcai_scores) > 0.5,
                         color="red", alpha=0.3)
    axes[2].set_ylabel("DTCAI Score")
    axes[2].set_title(f"DTCAI ({best_name}) = DTC * P(reliable)")
    axes[2].set_ylim(0, 1)
    for c in crashes:
        axes[2].axvline(df_btc.index[c.peak_date_idx], color="red",
                        linestyle="--", alpha=0.5)

    # Reliability vs DTC breakdown
    axes[3].plot(dtcai_dates, reliability_scores, color="green",
                 linewidth=0.8, alpha=0.7, label="P(reliable)")
    axes[3].plot(dtcai_dates, dtc_scores, color="orange",
                 linewidth=0.8, alpha=0.7, label="DTC")
    axes[3].set_ylabel("Score")
    axes[3].set_title("DTCAI Components")
    axes[3].set_ylim(0, 1)
    axes[3].legend()
    for c in crashes:
        axes[3].axvline(df_btc.index[c.peak_date_idx], color="red",
                        linestyle="--", alpha=0.5)

    plt.tight_layout()
    plt.show()

## 7. DTCAI vs LPPLS comparison table

For each known crash, compare the DTCAI score and raw LPPLS confidence
in the 60 days before the peak.

In [ ]:
if best_name:
    comparison = []
    for c in crashes:
        peak_date = df_btc.index[c.peak_date_idx]

        # LPPLS confidence 60d before peak
        lppls_mask = (
            (df_btc.index >= peak_date - pd.Timedelta(days=60))
            & (df_btc.index <= peak_date)
        )
        lppls_idx = np.where(lppls_mask)[0]
        lppls_vals = conf_arr[lppls_idx[lppls_idx < len(conf_arr)]]
        lppls_vals = lppls_vals[~np.isnan(lppls_vals)]

        # DTCAI 60d before peak
        dtcai_mask = (
            (dtcai_dates >= peak_date - pd.Timedelta(days=60))
            & (dtcai_dates <= peak_date)
        )
        dtcai_vals = dtcai_scores[dtcai_mask]
        dtcai_vals = dtcai_vals[~np.isnan(dtcai_vals)]

        comparison.append({
            "crash": f"{peak_date.date()} ({c.drawdown:.0%})",
            "lppls_max": lppls_vals.max() if len(lppls_vals) > 0 else np.nan,
            "lppls_mean": lppls_vals.mean() if len(lppls_vals) > 0 else np.nan,
            "dtcai_max": dtcai_vals.max() if len(dtcai_vals) > 0 else np.nan,
            "dtcai_mean": dtcai_vals.mean() if len(dtcai_vals) > 0 else np.nan,
        })

    comp_df = pd.DataFrame(comparison)
    print("60-day pre-crash signal comparison:")
    display(comp_df.style.format({
        "lppls_max": "{:.3f}", "lppls_mean": "{:.3f}",
        "dtcai_max": "{:.3f}", "dtcai_mean": "{:.3f}",
    }))

## 8. Feature importance (Random Forest)

If a Random Forest was trained, examine which LPPLS parameters
are most predictive of reliable fits.

In [ ]:
if "RF" in models:
    rf = models["RF"]
    importances = rf.model.feature_importances_

    fig, ax = plt.subplots(figsize=(8, 4))
    idx = np.argsort(importances)[::-1]
    ax.bar(range(7), importances[idx], color="steelblue")
    ax.set_xticks(range(7))
    ax.set_xticklabels([FEATURE_NAMES[i] for i in idx])
    ax.set_ylabel("Feature Importance")
    ax.set_title("Random Forest: Which LPPLS Parameters Predict Reliability?")
    plt.tight_layout()
    plt.show()

    for i in idx:
        print(f"  {FEATURE_NAMES[i]:>8s}: {importances[i]:.3f}")
else:
    print("RF model not available. Install scikit-learn: pip install fatcrash[deep]")

## Summary

### Key findings
- **Most LPPLS fits are unreliable** — the classifier learns to separate the
  signal from noise in the 7-dimensional parameter space.
- **DTCAI filters false positives**: by weighting the DTC ratio by P(reliable),
  the score is high only when both the timing is close *and* the parameters
  look historically credible.
- Random Forest often outperforms LogReg and is competitive with ANN,
  with the advantage of interpretable feature importances.
- The **tc** and **omega** parameters tend to be the most discriminative
  features for separating reliable from unreliable fits.

### Limitations
- Training and prediction are done on the same asset (BTC). Cross-asset
  generalization remains an open question.
- The label definition (tc within 10 days of actual crash) is sensitive
  to the crash detection criterion.
- Class imbalance (few reliable fits) limits precision.

### Connection to the aggregator
The DTCAI score feeds into the aggregator as `dtcai_signal` with weight 0.06
(see `fatcrash.aggregator.signals.DEFAULT_WEIGHTS`). It provides an
independent AI-based assessment complementing the traditional LPPLS
confidence indicator.

*All numbers are in-sample on historical data. This is not financial advice.*